## $$\textit{import Package}$$

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
%matplotlib Qt5
from data import *

a = np.linspace(0.5 / 5, 0.5, 4)
print(a)

## $$\textit{Initialization}$$

In [ ]:
np.random.seed(2)
np.set_printoptions(threshold = np.inf, linewidth = np.inf)

## $$\textit{Graph Setting}$$

In [ ]:
fig = plt.figure(figsize=[10,4], dpi=200)
ax1, ax2 = fig.subplots(1, 2)
print(type(ax1))

In [ ]:
state = state_generator(
    nbodies = 2, 
    mass = 1.0,
    min_radius = 1.0,
    max_radius = 2.0,
    orbit_noise = 0.05
)
orbit, settings = get_orbit(state, t_start = 0., t_step = 1000, interval = 0.05, rtol = 1e-12)

p1: np.ndarray = orbit[..., 0, 1:3]
p2: np.ndarray = orbit[..., 1, 1:3]

pax1, pay1 = np.split(p1, 2, axis = -1)
pax1, pay1 = pax1.squeeze(axis = 1), pay1.squeeze(axis = 1)
pax2, pay2 = np.split(p2, 2, axis = -1)
pax2, pay2 = pax2.squeeze(axis = 1), pay2.squeeze(axis = 1)

print(pax1.shape, pax2.shape, pay1.shape, pay2.shape)
x1, y1, x2, y2 = [], [], [], []

x_min = min(pax1.min(), pax2.min())
x_max = max(pax1.max(), pax2.max())
y_min = min(pay1.min(), pay2.min())
y_max = max(pay1.max(), pay2.max())

ax1.plot(pax1, pay1, 'o', markersize = 3)
ax1.plot(pax2, pay2, 'o', markersize = 3)

sub1_line1, = ax1.plot(x1, y1, 'o', animated = True, markersize = 7)
sub1_line2, = ax1.plot(x2, y2, 'o', animated = True, markersize = 7)
print(type(sub1_line1), type(sub1_line2))
print(orbit.shape)

In [ ]:
potential = potential_energy(orbit)
kinetic = kinetic_energy(orbit)
total = total_energy(orbit)
# print(total)
# np.savetxt('total_energy', total)
times = settings['t_eval']
print(potential.shape, kinetic.shape, total.shape, times.shape)
pe, ke, tote, t = [], [], [], []

# pe_line, ke_line, tote_line = ax2.plot(pe, t, ke, t, tote, t, 'o', animated = True)
ax2.plot(times, potential, 'o', markersize = 3)
ax2.plot(times, kinetic, 'o', markersize = 3)
ax2.plot(times, total, 'o', markersize = 3)
pe_line, = ax2.plot(t, pe, 'o', animated = True, markersize = 7)
ke_line, = ax2.plot(t, ke, 'o', animated = True, markersize = 7)
tote_line, = ax2.plot(t, tote, 'o', animated = True, markersize = 7)
tmin, tmax = times.min(), times.max()

e_min = min(potential.min(), kinetic.min(), total.min(), 0)
e_max = max(potential.max(), kinetic.max(), total.max())

print(f'min energy: {e_min}\nmax energy: {e_max}')
print(type(pe_line), type(ke_line), type(tote_line))

print(pax1.shape, pay1.shape, pax2.shape, pay2.shape, potential.shape, kinetic.shape, total.shape, times.shape)

In [ ]:
frames = np.stack([pax1, pay1, pax2, pay2, potential, kinetic, total, times], axis = 1)
timesteps = 1
frames = frames[::timesteps]
def init():
    qscale = 0.5
    escale = 0.5

    # coord initialization
    ax1.set_xlabel('$q_{x}$') ; ax1.set_ylabel('$q_{y}$')
    ax1.set_axis_on()
    ax1.set_xlim(x_min - qscale * abs(x_min), x_max + qscale * abs(x_max))
    ax1.set_ylim(y_min - qscale * abs(y_min), y_max + qscale * abs(y_max))
    ax1.set_title('Trajectories')

    # energy initialization
    ax2.set_axis_on()
    ax2.set_title("Energy")
    ax2.set_xlabel('time')
    ax2.set_xlim(tmin - 1.0, tmax + 1.0)
    ax2.set_ylim(e_min - escale * abs(e_min), e_max + escale * abs(e_max))

    ax1.legend([sub1_line1, sub1_line2], ['body 1 path','body 2 path'],fontsize = 8)
    ax2.legend([pe_line, ke_line, tote_line], ['potential', 'kinetic', 'total'], fontsize = 8)

    return sub1_line1, sub1_line2, pe_line, ke_line, tote_line


def update(frame, sub1_line1, sub1_line2, pe_line, ke_line, tote_line):

    px1, py1, px2, py2, p, k, tt, t_interval = frame
    # print(px1, py1, px2, py2, p, k, tt, t_interval)
    
    # update coord
    # x1.append(px1);y1.append(py1)
    # x2.append(px2);y2.append(py2)
    # if len(x1) > 580:
    #     x1.pop(0);y1.pop(0);x2.pop(0);y2.pop(0)
    # sub1_line1.set_data(x1, y1)
    # sub1_line2.set_data(x2, y2)

    sub1_line1.set_data(px1, py1)
    sub1_line2.set_data(px2, py2)

    # # update energy
    # pe.append(p);ke.append(k)
    # tote.append(tt);t.append(t_interval)
    # if len(pe) > 580:
    #     pe.pop(0);ke.pop(0);tote.pop(0);t.pop(0)
    # pe_line.set_data(t, pe)
    # ke_line.set_data(t, ke)
    # tote_line.set_data(t, tote)

    pe_line.set_data(t_interval, p)
    ke_line.set_data(t_interval, k)
    tote_line.set_data(t_interval, tt)

    ax1.legend([sub1_line1, sub1_line2], ['body 1 path','body 2 path'], fontsize = 8)
    ax2.legend([pe_line, ke_line, tote_line], ['potential', 'kinetic', 'total'], fontsize = 8)
    return sub1_line1, sub1_line2, pe_line, ke_line, tote_line

## $$\textit{Graph Generation}$$

In [ ]:
ani = animation.FuncAnimation(fig = fig, func = update, frames = frames, fargs = [sub1_line1, sub1_line2, pe_line, ke_line, tote_line], init_func = init, interval = 1, blit = True)
plt.show()

In [ ]:
ani.save('twobody.gif', writer = 'imagemagick', dpi = 100, progress_callback = lambda i, n: print(f'Saving frame {i}/{n}', end = '\r'))
# print(orbit)